In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
import zipfile
 # Replace 'DATASET.zip' with the actual name of your zip file
with zipfile.ZipFile('Dataset.zip', 'r') as zip_ref:     # replace it with ur zipfilename
    zip_ref.extractall('DATASET') # extracts to a directory called 'DATASET'

In [3]:
# Paths to dataset folders
TRAIN_DIR = '/content/DATASET/DATASET/Train'
TEST_DIR = '/content/DATASET/DATASET/Test'
#VID_DIR = '/content/DATASET/PLD_3_Classes_256/Validation'

  # Image size and batch size
IMG_SIZE = (224, 224)  # Resize all images to 128x128
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
#validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'   # categorical
 )
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  #categorical
 )

Found 1106 images belonging to 3 classes.
Found 218 images belonging to 3 classes.


In [4]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224,224, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')  # softmax for multi classification
    ])
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',  #sparse_categorical_crossentropy
                  metrics=['accuracy'])
    return model
model = build_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
 )

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 7/35 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.3236 - loss: 3.3292

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


35/35 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.4037 - loss: 1.9473 - val_accuracy: 0.2294 - val_loss: 1.1736
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 33s 800ms/step - accuracy: 0.5344 - loss: 0.9887 - val_accuracy: 0.3807 - val_loss: 1.1000
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 32s 778ms/step - accuracy: 0.5860 - loss: 0.9364 - val_accuracy: 0.3807 - val_loss: 1.3237
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 40s 786ms/step - accuracy: 0.5860 - loss: 0.9301 - val_accuracy: 0.4908 - val_loss: 0.9691
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 42s 782ms/step - accuracy: 0.6346 - loss: 0.8008 - val_accuracy: 0.6009 - val_loss: 0.8047
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 772ms/step - accuracy: 0.7100 - loss: 0.7407 - val_accuracy: 0.6789 - val_loss: 0.6775
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 800ms/step - accuracy: 0.7570 - loss: 0.6183 - val_accuracy: 0.7477 - val_loss: 0.5939
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 32s 760ms/step - accuracy: 0.8000 - loss: 0.4883 - val_accuracy: 0.8303 -

In [6]:
 # Saving model
model.save('Good_Food_classify.h5')


In [7]:
 # 4. Evaluate the Model on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 710ms/step - accuracy: 0.8423 - loss: 0.3924
Test Accuracy: 0.8623853325843811


In [9]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')
 # Load the trained model
model = load_model('/content/Good_Food_classify.h5')  # Updated model path
 # Path to your custom image
image_path = r"/content/Img2 (6).jpg"  # Replace with your image path
 # Preprocess the image
IMG_SIZE = (224, 224)  # Must match the size used during training
def preprocess_image(image_path):
  img = load_img(image_path, target_size=IMG_SIZE)  # Load and resize image
  img_array = img_to_array(img)  # Convert image to array
  img_array = img_array / 255.0  # Rescale to 0-1 range
  img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
  return img_array
image = preprocess_image(image_path)
 # Make prediction
prediction = model.predict(image)
 # Class labels (ensure they match the model's training)
class_labels = ['fruit_images', 'party_food_images', 'veggie_images']
 # Get the predicted class index
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class_label = class_labels[predicted_class_index]
 # Output the prediction
print(f"The image is predicted to be: {predicted_class_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
The image is predicted to be: veggie_images
